In [1]:
# IMPORTANTE:
# não se esqueça de atualizar as datas [filtros e excel]
# sempre a ultima versão do banco analisada
# ULTIMA ANÁLISE: 03/03/2022

In [1]:
import pandas as pd
import datetime
import warnings
warnings.filterwarnings("ignore")

In [45]:
# puxar arquivo do banco
df = pd.read_csv('eapv_03032022_09h07.csv', sep=';', encoding='utf-8')

print(f'Banco importado: {df.shape}')
banco_completo = df.shape[0]

Banco importado: (17756, 189)


# Excluir as fichas canceladas

In [46]:
# Transformando ['Número da Notificação'] para string
df['Número da Notificação'] = df['Número da Notificação'].astype(str)

In [47]:
# puxar arquivo canceladas
df_canceladas = pd.read_csv('eapv_canceladas_03032022.csv', sep=';', encoding='utf-8')

# df_canceladas passa a ser apenas a coluna 'Número da Notificação' e em string
df_canceladas = df_canceladas[['Número da Notificação']].astype(str)
print(f'Banco canceladas: {df_canceladas.shape}')
banco_canceladas = df_canceladas.shape[0]

# Transformando o banco df_canceladas para uma lista notif_canceladas
notif_canceladas = df_canceladas['Número da Notificação'].to_list()
#notif_canceladas

Banco canceladas: (800, 1)


In [48]:
# TIRANDO AS EXCLUÍDAS DO BANCO DE DADOS 
df = df[~df['Número da Notificação'].str.contains('(?i)' + '|'.join(notif_canceladas), na=False)]

# conferir
print(f'Banco sem as canceladas: {df.shape[0]}')
print(f'Banco completo: {banco_completo} - canceladas: {banco_canceladas} = {banco_completo - banco_canceladas}')

Banco sem as canceladas: 16956
Banco completo: 17756 - canceladas: 800 = 16956


In [49]:
# Verificar se ficou alguma cancelada e excluir do banco
df[df['Situação da Notificação'] == 'Cancelado']
df = df[~df['Situação da Notificação'].str.contains('Cancelado', na=False)]

# Tratar vacinas

In [50]:
# Criar df_dubias apenas com as dubias
df_dubias = df.loc[~(df['Imunobiológico (vacina)'].str.contains('(?i)Butantan') |
             df['Imunobiológico (vacina)'].str.contains('(?i)Sinovac') |
             df['Imunobiológico (vacina)'].str.contains('(?i)Oxford') |
             df['Imunobiológico (vacina)'].str.contains('(?i)AstraZeneca') |
             df['Imunobiológico (vacina)'].str.contains('(?i)Covishield') |
             df['Imunobiológico (vacina)'].str.contains('(?i)Pfizer') |
             df['Imunobiológico (vacina)'].str.contains('(?i)Janssen'))]

# gerar excel
df_dubias.to_excel('vacinas_dubias.xlsx')

In [51]:
# Manter no df apenas vacinas corretas 
df = df.loc[(df['Imunobiológico (vacina)'].str.contains('(?i)Butantan') |
             df['Imunobiológico (vacina)'].str.contains('(?i)Sinovac') |
             df['Imunobiológico (vacina)'].str.contains('(?i)Oxford') |
             df['Imunobiológico (vacina)'].str.contains('(?i)AstraZeneca') |
             df['Imunobiológico (vacina)'].str.contains('(?i)Covishield') |
             df['Imunobiológico (vacina)'].str.contains('(?i)Pfizer') |
             df['Imunobiológico (vacina)'].str.contains('(?i)Janssen'))]

In [52]:
bd_vacinas_covid = df.shape[0]

print(f'Foram excluídas do banco {banco_completo - bd_vacinas_covid} fichas que contém outras vacinas que não as COVID-19')
print(f'Banco somente com vacinas COVID-19: {df.shape}')

Foram excluídas do banco 924 fichas que contém outras vacinas que não as COVID-19
Banco somente com vacinas COVID-19: (16832, 189)


.

.

A PARTIR DAQUI, É POSSÍVEL RODAR SOMENTE AS CÉLULAS DE INTERESSE

# Abaixo de 12 anos

In [22]:
# Encontrar novos casos e enviar para [helena-ramos@saude.rs.gov.br]
df_12_anos = df.copy()

In [23]:
# Filtrar menor que 12 anos
df_12_anos = df_12_anos[df_12_anos['Idade Evento'] < 12]
# Colocar em ordem crescente
df_12_anos = df_12_anos.sort_values('Idade Evento').reset_index(drop=True)
# Dropar valores nulos na coluna 'Nome Completo'
df_12_anos.dropna(subset=['Nome Completo'], inplace=True)
# Gerar excel com total [OPCIONAL]
df_12_anos.to_excel('eapv_abaixo_12_anos_total.xlsx')

In [20]:
## ALTERAR DATA
# Filtrar apenas as notificaçoes novas
#df_12_anos['Data da Notificação'] = pd.to_datetime(df_12_anos['Data da Notificação'])
df_12_anos['Data da Notificação'] =  pd.to_datetime(df_12_anos['Data da Notificação'], infer_datetime_format=True, format='%d/%m/%Y')
df_12_anos = df_12_anos[df_12_anos['Data da Notificação']>= '2022-02-08']
# Filtrar graves
df_12_anos = df_12_anos.loc[df_12_anos['Classificação de gravidade'].str.contains("Grave", na=False)]

In [21]:
# Consultar casos
print(f'Há {df_12_anos.shape[0]} casos')
# Gerar excel
#df_12_anos = df_12_anos[['Número da Notificação', 'Nome Completo', 'Idade Evento']]
#df_12_anos.to_excel('eapv_abaixo_12_anos_03032022.xlsx')

Há 228 casos


# Guillain Barre

In [14]:
# Encontrar novos casos e enviar para [rosemeri-cohen@saude.rs.gov.br]
df_guillain = df.copy()

In [15]:
# pesquisa 'guill' em todo o dataframe
df_guillain = df_guillain.loc[df_guillain.apply(lambda row: row.astype(str).str.contains('(?i)guill').any(), axis=1)]

In [16]:
## ALTERAR DATA
# Filtrar apenas as notificaçoes novas
df_guillain['Data da Notificação'] = pd.to_datetime(df_guillain['Data da Notificação'], infer_datetime_format=True, format='%d/%m/%Y')
df_guillain = df_guillain[df_guillain['Data da Notificação']>= '2022-02-08']

In [17]:
# Consultar se há novos casos antes de gerar excel
print(f'Há {df_guillain.shape[0]} casos')
# Gerar excel com data recorrente
#df_guillain = df_guillain[['Número da Notificação', 'Nome Completo']]
#df_guillain.to_excel('eapv_guillain_barre_03032022.xlsx')

Há 0 casos


# Tromboses

In [56]:
# Encontrar novos casos e enviar para [rosemeri-cohen@saude.rs.gov.br]
df_tromboses = df.copy()

In [57]:
# Fazer um lista com os nomes para pesquisa 
tromboses = ['trombo', 'embolia']
# Pesquisa {tromboses} em todo o dataframe
df_tromboses = df_tromboses.loc[df_tromboses.apply(lambda row: row.astype(str).str.contains('(?i)' + '|'.join(tromboses)).any(), axis=1)]

In [58]:
## ALTERAR DATA
# Filtrar apenas as notificaçoes novas
df_tromboses['Data da Notificação'] = pd.to_datetime(df_tromboses['Data da Notificação'], infer_datetime_format=True, format='%d/%m/%Y')
df_tromboses = df_tromboses[df_tromboses['Data da Notificação']>= '2022-02-08']

In [59]:
# Consultar se há novos casos antes de gerar excel
print(f'Há {df_tromboses.shape[0]} casos')
# Gerar excel com data recorrente
df_tromboses = df_tromboses[['Número da Notificação', 'Nome Completo']]
df_tromboses.to_excel('eapv_tromboses_03032022.xlsx')

Há 18 casos


# Gestantes

In [75]:
# Encontrar novos casos e enviar para [helena-ramos@saude.rs.gov.br]
df_gestantes = df.copy()

In [76]:
df_gestantes = df_gestantes.loc[df_gestantes['Gestante no momento da vacinação?'].str.contains("(?i)Sim", na=False)]
df_gestantes = df_gestantes.loc[df_gestantes['Classificação de gravidade'].str.contains("Grave", na=False)]

In [77]:
# Filtrar apenas as notificaçoes novas / SEMPRE ALTERAR A DATA
df_gestantes['Data da Notificação'] = pd.to_datetime(df_gestantes['Data da Notificação'], infer_datetime_format=True, format='%d/%m/%Y')
df_gestantes = df_gestantes[df_gestantes['Data da Notificação']>= '2022-02-08']

In [79]:
# Consultar se há novos casos antes de gerar excel
print(f'Há {df_gestantes.shape[0]} casos')
# Gerar excel com data recorrente
df_gestantes = df_gestantes[['Número da Notificação', 'Nome Completo']]
df_gestantes.to_excel('eapv_gestantes_graves_03032022.xlsx')

Há 1 casos


# Encerrados graves por municípios

In [ ]:
# bug no e-sus, mostra encerrados pelo estado na coluna dos municípios... verificar manualmente

In [80]:
# Encontrar novos casos e enviar para [helena-ramos@saude.rs.gov.br]
df_municipio = df.copy()

In [81]:
df_municipio = df_municipio.loc[df_municipio['Classificação de gravidade'].str.contains("Grave", na=False)]
df_municipio.dropna(subset=['Encerramento Municipal - Evento adverso'], inplace=True)

In [82]:
# Filtrar apenas as notificaçoes novas / SEMPRE ALTERAR A DATA
df_municipio['Data da Notificação'] = pd.to_datetime(df_municipio['Data da Notificação'], infer_datetime_format=True, format='%d/%m/%Y')
df_municipio = df_municipio[df_municipio['Data da Notificação']>= '2022-02-08']

In [83]:
# Consultar se há novos casos antes de gerar excel
print(f'Há {df_municipio.shape[0]} casos')
# Gerar excel com data recorrente
#df_municipio.to_excel('encerrados_por_municipio.xlsx')

Há 0 casos


In [ ]:
# PARA FAZER
# Filtrar apenas colunas de interesse. Identificar quais...
# Verificar casos de SARC em Imunobiológico(vacina) e gerar xlsx